In [1]:
from utils import load_data
import numpy as np
import models 

In [2]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [3]:
# np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, adj, T, group_num = load_data(1,data_file)

[list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
 list([13, 14, 15, 16, 17]) list([18, 19, 20, 21, 22, 23, 24])
 list([25, 26, 27, 28, 29, 30, 31, 32, 33])
 list([34, 35, 36, 37, 38, 39, 40, 41, 42])
 list([43, 44, 45, 46, 47, 48, 49, 50]) list([51, 52, 53, 54, 55, 56, 57])
 list([58, 59, 60, 61, 62, 63]) list([64, 65, 66, 67, 68, 69, 70, 71])
 list([72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82])
 list([83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93])
 list([94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104])
 list([105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])
 list([117, 118, 119, 120, 121, 122, 123, 124, 125, 126])
 list([127, 128, 129, 130, 131, 132, 133, 134])
 list([135, 136, 137, 138, 139, 140, 141, 142, 143])
 list([144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155])
 list([156, 157, 158, 159, 160, 161, 162, 163, 164, 165])
 list([166, 167, 168, 169, 170, 171, 172, 173, 174, 175])
 list([176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188])
 list([1

D:\zhuomian\file\弱网中鲁棒性的恶意节点检测\exp\utils.py:389: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:653.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [4]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(T)
# print(adj[-1])

In [5]:
# 打印边连接关系（processed_data.edge_index）
print(processed_data[-1].edge_index.numpy().size)

154


In [6]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
print(processed_data[-1].x[:,0:8].numpy().size)

616


In [7]:
train_model = models.HypergraphModel(8, 3)
print(train_model)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(8, 64)
  )
  (layer2): HypergraphConvLayer(
    (conv): HypergraphConv(64, 32)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=2, bias=True)
  )
  (norm1): CustomBatchNorm()
  (norm2): CustomBatchNorm()
  (norm3): CustomBatchNorm()
)


In [8]:
import time
import torch.optim as optim
from utils import accuracy
import torch.nn.functional as F
import torch
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
def train(epoch):
    t = time.time()
    train_model.train()
    
    
    optimizer.zero_grad()
    # print(processed_data[0].x[0:8].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(group_num):
        print(i)
        output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, 
                                 processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
        # print(output)
        labels = processed_data[1].y
        loss_train = criteria(output, labels)

        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()
        
        train_model.eval()
        output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, 
                                 processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
        # print(output)
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [9]:
def test():
    train_model.eval()
    output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, 
                             processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
    labels = processed_data[0].y
    loss_test = criteria(output, labels)
    acc_test = acc_measure(labels, output)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    return acc_test.item()

In [10]:
# torch.autograd.set_detect_anomaly(True)
num_epochs = 100
early_stopping = 30
val_watch = []
for epoch in range(num_epochs):
    val_watch.append(train(epoch))
    test()
    if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
        print("Early stopping...")
        break
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")

0
loss_train: 0.7840 acc_train: 0.1000 loss_val: 0.7486 acc_val: 0.9000 time: 0.0195s
1


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [64, 32]], which is output 0 of AsStridedBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).